In [ ]:
%reset

In [ ]:
!pip install statsmodels

In [ ]:
import datetime
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.tsa as tsa
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tools.sm_exceptions import ConvergenceWarning

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input , Dropout , Dense , RepeatVector , TimeDistributed , Flatten , LSTM
from tensorflow.keras import Model
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import MinMaxScaler , StandardScaler

import warnings
warnings.filterwarnings('ignore' , category=RuntimeWarning)
warnings.simplefilter('ignore' , ConvergenceWarning)
warnings.simplefilter('ignore' , UserWarning)



In [ ]:
# global variables
#_______________________________________________________________________________________

# RESAMPLED EVERY SIGNAL TO THIS RATE OF CONSISTENCY
BASIC_SRATE = 128
print('Basic Sampling Rate : ',BASIC_SRATE)

# ---------------------------PARENT DB DIRECTORY----------------------------------------

DB_NPY = '../input/ecg-lead-2-dataset-physionet-open-access/db_npy'
print('DATASET DIR ::',DB_NPY)

# >------ load standard labels ------------------------------------------
std_annotations = os.path.join(DB_NPY,'annotations.txt')
print('STD ANN FILE ::',std_annotations)
std_ant = np.loadtxt(std_annotations, dtype='str',delimiter="\t")

# >------ load standard labels ------------------------------------------
std_annotations = os.path.join(DB_NPY,'annotations.txt')
print('STD ANN FILE ::',std_annotations)
std_ant = np.loadtxt(std_annotations, dtype='str',delimiter="\t")

#-----------------------------------------------------------------------
# seperate out beat and non beat annotations using 'annotations.txt'

beat_ants = []
non_beat_ants = []
print('\n Standard Labels')
for a  in std_ant:
    print(a[0] + '\t'+a[1] + '\t'+a[2])
    if a[1] == 'b': ##  is a beat anntations
        beat_ants.append(a[0])
    else:
        non_beat_ants.append(a[0])
print('beat annotations : ' , len(beat_ants))
print('non-beat annotations : ' , len(non_beat_ants))
#----------------------------------------------------------------------------------------------
gbreak = '---------------------------------------------------'
# data file identifiers
g_BEAT = '_BEAT.npy'       # beat annotations
g_NBEAT = '_NBEAT.npy'     #  non-beat annotations
g_SIG2 = '_SIG_II.npy'     # Signal Data (Removed manual gain)

In [ ]:
#<< --- define custom labels. 
_N = '_N'  # N - normal class - Non-Ectopic
_S = '_S'  # S - SupraVentricular Ectopic Beats (SVEBs)
_V = '_V'  # V - Ventricular Ectopic Beats (VEBs)
_F = '_F'  # F - Fusion Beats
_Q = '_Q'  # Q - Unrecognized
_X = '_X'  # X - Unmapped - if you don't want to use a lable, map it under this class - it should not be used in classification

custom_ants = [ _N, _S, _V, _F, _Q ]  # as recomended by AAMI

# define a mapping dictionary #----------------------------------
custom_cols = { _N:'tab:green',
                _S:'tab:red',
                _V:'tab:blue',
                _F:'tab:purple',
                _Q:'yellow',
                _X:'tab:gray'}
custom_mapping = {
                    #<--- Normal Class
                    'N': _N, 
                    'L': _N,
                    'R': _N,
                    'B': _N,
    
                    #<--- SVEB
                    'A': _S,
                    'a': _S,
                    'J': _S,
                    'S': _S,
    
                    #<--- VEB
                    'V': _V, 
                    'r': _V,
    
                    #<--- FUSION
                    'F': _F,
    
                    #<---* Supraventricular escape - aami says its normal but should be mapped to _S
                    'e': _S,
                    'j': _S,
                    'n': _S,
    
                    #<---* Ventricular escape - aami says its normal but should be mapped to _V
                    'E': _V,
    
                    #<--- Paced beats are unmapped - dont use record containing paced beats (mitdb - 102,104,107,217)
                    'f': _X,
                    '/': _X,
    
                    #<--- Unrecognised or Unclassifiable
                    'Q': _Q,
                    '?': _Q,
                }

In [ ]:
# class definitions

class ecgDatabase:
    def __init__(self, db_name, db_path):
        self.db_name = db_name
        self.db_path = db_path
        self._prepare()
        
    def _prepare(self):
        # 1. read record list
        self.record_list = np.loadtxt(os.path.join(self.db_path,'RECORDS'), dtype='str')
        if len(self.record_list.shape)==0:
            self.record_list = np.array([self.record_list])
        
        # 2. prepare a dictionary of ecgRecord objects
        self.records = {}
        for irec in self.record_list:
            self.records[irec] = ecgRecord(irec,os.path.join(self.db_path),self.db_name) 
            #<--- records are not loaded initially, will be loaded dynamically
        return
        
    def get_record(self, rec_name):
        if rec_name in self.records.keys():
            this_rec = self.records[rec_name]
            if not this_rec.isloaded:
                this_rec.loadfromdisk()  #<--- Loads records dynamically, on call to this function
            return this_rec
        else:
            return None  #<--- No record with name <rec_name>
        
#-------------------------------------------------------------------------------------------------------

class ecgRecord:
    def __init__(self , arg_recname , arg_path , db_name):
        self.dbname = db_name
        self.path = arg_path
        self.name = arg_recname
        self.isloaded = False
        
#------------------------------------------------------------------------------------------------------

    def loadfromdisk(self , verbose = 0):
        if verbose > 0 :
            print('\nLoading Record' , self.name)
        #------------------------------------------------------------------------------------------
        # Load Signal Data
        #------------------------------------------------------------------------------------------
        
        self.signal_file = os.path.join(self.path , self.name + g_SIG2)
        self.iSignal = np.load(self.signal_file)
        if verbose > 0:
            print('Signal Length',self.iSignal.shape)
            print('Signal Duration',len(self.iSignal)/BASIC_SRATE,'sec')
        
        #------------------------------------------------------------------------------------------
        
        # Load-Beat Annotation
        self.beat_file = os.path.join(self.path,self.name + g_BEAT)
        self.iBeat = np.load(self.beat_file)
        if verbose>0:
            print('#Beats : ' , self.iBeat.shape)
        
        # seperate out R-peak and Labels(note both are in string format)
        self.iRpeaks = self.iBeat[:,0].astype('int')
        self.iLabels = self.iBeat[:,1]
        self.nos_beats= len(self.iRpeaks)
        
        self.isloaded = True
        return self.isloaded
    
    #------------------------------------------------------------------------------------------------
    def map_beat_labels(self , c_mapping , c_colors):
        self.mLabels = np.zeros(len(self.iLabels),dtype = 'U2')
        self.mColors = np.zeros(len(self.iLabels) , dtype = 'U10')
        for i in range(0,len(self.iLabels)):
            self.mLabels[i] = c_mapping[self.iLabels[i]]
            self.mColors[i] = c_colors[self.mLabels[i]]
        return len(self.mLabels)
    
    #------------------------------------------------------------------------------------------------
    
    def get_true_range(self,pf,pt):

        N = len(self.iSignal)
        if pt<pf:
            pt,pf = pf,pt # swap
            
        if pf<0:
            pf=0
        elif pf>=N:
            pf=N-1
            
        if pt<0:
            pt=0
        elif pt>N:
            pt=N
        return pf, pt
    
    #------------------------------------------------------------------------------------------------
    
    def get_signal_slice_fd(self,pf, pt):
        signal_slice = self.iSignal[pf:pt]
        signal_list = np.where((self.iRpeaks>=pf) & (self.iRpeaks<pt))[0]
        return signal_slice, signal_list, pf, pt
    
    #------------------------------------------------------------------------------------------------
    
    def get_signal_slice_biV(self, beat_index, minus_i, plus_i ,signal_trim_left, signal_trim_right ):
        RpeakL = self.iRpeaks[beat_index-minus_i]+signal_trim_left
        RpeakC = self.iRpeaks[beat_index]
        RpeakR = self.iRpeaks[beat_index+plus_i]-signal_trim_right
        pf,pt = RpeakL,RpeakR
        signal_slice=self.iSignal[pf:pt]
        signal_list = np.where((self.iRpeaks>=pf) & (self.iRpeaks<pt))[0]
        return signal_slice, signal_list, pf, pt
    #---------------------------------------------------------------------------------------------
    
    def get_signal_slice_biF(self, beat_index, left_samples, right_samples ):
        Rpeak = self.iRpeaks[beat_index]
        pf, pt =  Rpeak-left_samples, Rpeak+right_samples
        signal_slice = self.iSignal[pf:pt]
        signal_list = np.where((self.iRpeaks>=pf) & (self.iRpeaks<pt))[0]
        return signal_slice, signal_list, pf, pt
    
    #---------------------------------------------------------------------------------------------
        
    
    
        
        

In [ ]:
yll, yul, ycl = -3, 3.5, -2.8

def plot_beat(signal_slice, signal_list, pf, pt, ph,
                show_label_ticks=False, show_custom_labels=True, show_custom_vline=False,
                w=0.012, h=4, r=0.5):
    plt.figure(figsize=(w*len(signal_slice),h), constrained_layout=True)
    plt.ylim(yll, yul)
    plt.xlim(pf,pt)
    
    ilen = len(signal_list)
    _range = np.arange(pf,pt,BASIC_SRATE)
    _peaks = rec.iRpeaks[signal_list]
    _labels = rec.iLabels[signal_list]
    _colorL = rec.mColors[signal_list]
    
    # plot the signal
    plt.plot(np.arange(pf,pt,1), signal_slice, color='black',linewidth=0.8)
    plt.vlines(ph,yll,yul,linewidth=0.8,color='black' )
    # plot ticks
    if show_label_ticks:
        plt.xticks(_peaks,_labels) #-pf*BASIC_SRATE
        plt.grid( axis='x')
    else:
        plt.xticks(_range,np.round(_range/BASIC_SRATE,2))#,rotation=60)

    # plot labels
    if show_custom_labels:
        plt.scatter(_peaks ,np.zeros(ilen)+ycl, c=_colorL)
        if show_custom_vline:
            for j in range(0,ilen):
                plt.annotate(signal_list[j],xy=(_peaks[j],ycl+0.3) )
                plt.vlines(_peaks[j],yll,yul,linewidth=0.4,color=_colorL[j])

    plt.show()
    return
    

In [ ]:
# all databases---------------------------------------------
db_list = ['mitdb','svdb', 'incartdb']
rf = open('count.csv','w')  # creates this csv, use it to investigate further
sep = ','  # comma seperated values

# prepare header---------------------------------------------
header = 'DATABASE'+sep+'RECORD'+sep+'#BEATS'
for sa in  beat_ants:
    header+= sep+sa
header+=sep+'SUM'
for ca in  custom_ants:
    header+= sep+ca
header+=sep+'CSUM'

header+=sep + "NN_ALL"
header+=sep + "NN_TRUE"
header+=sep + "NN_SEMI"
header+=sep + "NN_ABS"
header+=sep + "AA_ALL"
header+=sep + "SS_ALL"
header+=sep + "VV_ALL"
header+=sep + "FF_ALL"
header+=sep + "QQ_ALL"
header+=sep + "XX_ALL"

print(header)
rf.write(header+'\n')

# start loop for each db...
for sel_db_name in db_list:

    sel_db_path = os.path.join(DB_NPY, sel_db_name+'_npy')
    sel_db_files = os.listdir(sel_db_path)
    dbo = ecgDatabase(sel_db_name, sel_db_path)
    sel_recs = []
    for recn in dbo.record_list:
        res_r = sel_db_name
        _load_success=False
        try:
            rec = dbo.get_record(recn)
            _load_success = True
        except:
            _load_success = False
        if not _load_success:
            continue
        res_r += sep+rec.name+sep+ str(rec.nos_beats)
        nos_map = rec.map_beat_labels(custom_mapping, custom_cols)# <<-- custom lables
        saDict={}
        sums = 0
        for sa in  beat_ants:
            saDict[sa] = np.where(rec.iLabels==sa)[0]
            isum = len(saDict[sa])
            sums+=isum
            res_r+=sep+ str(isum)
            #print(sa,'\t\t',isum)
        res_r+=sep+ str(sums)
        caDict={}
        sums = 0
        for ca in  custom_ants:
            caDict[ca] = np.where(rec.mLabels==ca)[0]
            isum = len(caDict[ca])
            sums+=isum
            res_r+=sep+ str(isum)
        res_r+=sep+ str(sums)
        # RR-intervals  **NOTE:: SKIPPING FIRST AND LAST BEAT OF THE RECORD
        rp_p, rp_c, rp_n = rec.iRpeaks[0:-2], rec.iRpeaks[1:-1], rec.iRpeaks[2:]
        mLabels_p, mLabels_c, mLabels_n = rec.mLabels[0:-2],rec.mLabels[1:-1],rec.mLabels[2:]

        rri_prev = (rp_c-rp_p)                 #<-- previous rr-interval
        rri_next = (rp_n-rp_c)                 #<-- next rr-interval
        rri_mean = (rri_prev + rri_next)/2       #<-- mean of prev and next
        rri_delta = rri_next - rri_prev        #<-- delta b/w prev and next
        rri_delta_abs = np.abs(rri_delta)       #...... for a normal rhythm, rri_abs_delta should be very small

        ABS_DELTA_LIM = int(0.04*BASIC_SRATE)  #-- define a limit on rri_delta_abs for a beat to be considered Normal rhythm = 0.04 sec

        # Any Normal: All Normal beats
        NN_all = np.where((mLabels_c==_N))[0]+1

        # True Normal: a Normal beat surronded by normal beats on both sides
        NN_true = np.where((mLabels_p==_N) & (mLabels_c==_N) & (mLabels_n==_N))[0]+1

        # Semi Normal: Normal beat surronded by atleast one abnormal beat on any side
        NN_semi = np.where((mLabels_c==_N) & ((mLabels_p!=_N) | (mLabels_n!=_N)))[0]+1

        # Absolute Normal: True Normal and delta_abs_rri < ABS_DELTA_DUR
        NN_abs = np.where((mLabels_p==_N) & (mLabels_c==_N) & (mLabels_n==_N)
                          & (rri_delta_abs<=ABS_DELTA_LIM))[0] + 1

        # Abnormal beats
        AA_all =np.where((mLabels_c!=_N))[0] + 1
        SS_all = np.where((mLabels_c==_S))[0] + 1
        VV_all = np.where((mLabels_c==_V))[0] + 1
        FF_all = np.where((mLabels_c==_F))[0] + 1
        QQ_all = np.where((mLabels_c==_Q))[0] + 1
        
        # Unmapped beats
        XX_all = np.where((mLabels_c==_X))[0] + 1
    
        res_r+= (
            sep + str(len(NN_all)) + sep + str(len(NN_true)) + 
            sep + str(len(NN_semi)) + sep + str(len(NN_abs)) + sep + str(len(AA_all)) 
            + sep + str(len(SS_all))+ sep + str(len(VV_all)) + sep + str(len(FF_all))+ sep + str(len(QQ_all))+ sep + str(len(XX_all)))
        #if (len(NN_true>=min_N) and len(AA_all)>=min_A):
        #    sel_recs.append(rec.name)
        print(res_r)
        rf.write(res_r+'\n')
        
    #sel_rec_dict[sel_db_name] = sel_recs
    
rf.close()


In [ ]:
sel_db_name = 'mitdb'
sel_db_path = os.path.join(DB_NPY , sel_db_name + '_npy')
sel_db_files = os.listdir(sel_db_path)
print(sel_db_path , ',',len(sel_db_files),'files')

dbo = ecgDatabase(sel_db_name , sel_db_path)
print(dbo.record_list.shape,dbo.record_list)

In [ ]:
# Select a record

recn = '207'
rec = dbo.get_record(recn)
print(rec.name , '\t' + str(rec.nos_beats) + 'beats')
nos_map = rec.map_beat_labels(custom_mapping , custom_cols) # custom labels
print('Mapped Lables : ', nos_map)

In [ ]:
# Find count of all beats
print('Record:' , rec.dbname+'/'+rec.name , '\t' + str(rec.nos_beats) + 'beats')
print(gbreak)
print('Standard Label Count')
saDict = {}
sum = 0
for sa in beat_ants:
    saDict[sa] = np.where(rec.iLabels == sa)[0]
    isum = len(saDict[sa])
    sum+=isum
    print(sa, '\t\t',isum)
print('__','\t\t',sum)
print(gbreak)
print('\nCustom Label Count')
caDict = {}
sum = 0
for ca in custom_ants :
    caDict[ca] = np.where(rec.mLabels == ca)[0]
    isum = len(caDict[ca])
    sum+=isum
    print(ca,'\t\t',sum)
print('__','\t\t',sum)
print(gbreak)

#RR-interval

rp_p , rp_c , rp_n = rec.iRpeaks[0:-2] , rec.iRpeaks[1:-1],rec.iRpeaks[2:]
mLabels_p,mLabels_c,mLabels_n = rec.mLabels[0:-2],rec.mLabels[1:-1],rec.mLabels[2:]

rri_prev = (rp_c-rp_p)                 #<-- previous rr-interval
rri_next = (rp_n-rp_c)                 #<-- next rr-interval
rri_mean = (rri_prev + rri_next)/2       #<-- mean of prev and next
rri_delta = rri_next - rri_prev        #<-- delta b/w prev and next
rri_delta_abs = np.abs(rri_delta)       #...... for a normal rhythm, rri_abs_delta should be very small

ABS_DELTA_LIM = int(0.04*BASIC_SRATE)

# Any Normal: All Normal beats
NN_all = np.where((mLabels_c==_N))[0]+1

# True Normal: a Normal beat surronded by normal beats on both sides
NN_true = np.where((mLabels_p==_N) & (mLabels_c==_N) & (mLabels_n==_N))[0]+1

# Semi Normal: Normal beat surronded by atleast one abnormal beat on any side
NN_semi = np.where((mLabels_c==_N) & ((mLabels_p!=_N) | (mLabels_n!=_N)))[0]+1

# Absolute Normal: True Normal and delta_abs_rri < ABS_DELTA_DUR
NN_abs = np.where((mLabels_p==_N) & (mLabels_c==_N) & (mLabels_n==_N)
                  & (rri_delta_abs<=ABS_DELTA_LIM))[0] + 1

# Abnormal beats
AA_all =np.where((mLabels_c!=_N))[0] + 1 
SS_all =np.where((mLabels_c==_S))[0] + 1
VV_all =np.where((mLabels_c==_V))[0] + 1
FF_all =np.where((mLabels_c==_F))[0] + 1
QQ_all =np.where((mLabels_c==_Q))[0] + 1


# Unmapped beats
XX_all = np.where((mLabels_c==_X))[0] + 1

print("\nNormals\t\t", len(NN_all))
print(" True-type\t", len(NN_true))
print(" Semi-type\t", len(NN_semi))
print(" Abs-type\t", len(NN_abs))
print("\nAbnormals\t", len(AA_all))
print(" S-type\t\t", len(SS_all))
print(" V-type\t\t", len(VV_all))
print(" F-type\t\t", len(FF_all))
print(" Q-type\t\t", len(QQ_all))
print("\nUnmapped\t", len(XX_all))
print(gbreak)

# plotting

plt.figure('RRIs', figsize = (int(0.04 * rec.nos_beats),4),dpi = 150)
plt.xlim(0,rec.nos_beats + 50)
plt.xticks(AA_all , fontsize = 4 , rotation = 90)
plt.grid(axis = 'x')
xrng = np.arange(1,rec.nos_beats-1,1)
assert (len(xrng) == len(rri_mean))
plt.plot(xrng , rri_mean , linewidth = 0.8 , color = 'black',label = 'RRI_mean')
plt.plot(xrng,rri_delta_abs,linewidth = 0.8 , color = 'tab:blue',label  = 'RRI_delta')
plt.scatter(NN_all, np.zeros(len(NN_all)), color=custom_cols[_N], marker='.', label='Normal' )
plt.scatter(SS_all, np.zeros(len(SS_all))+5, color=custom_cols[_S], marker='.', label='S-Type' )
plt.scatter(VV_all, np.zeros(len(VV_all))+5, color=custom_cols[_V], marker='.', label='V-Type' )
plt.scatter(FF_all, np.zeros(len(FF_all))+5, color=custom_cols[_F], marker='.', label='F-Type' )
plt.scatter(QQ_all, np.zeros(len(QQ_all))+5, color=custom_cols[_Q], marker='.', label='Q-Type' )
plt.scatter(XX_all, np.zeros(len(XX_all))-5, color=custom_cols[_X], marker='.', label='Unmapped' )
plt.legend()
plt.show()


In [ ]:
## plot full record ECG

# each record has 1805 sec of ECG = 231111 samples

print('RECORD:',rec.dbname+'/'+rec.name)
sig_len = len(rec.iSignal)
sig_dur = sig_len/BASIC_SRATE
print('Signal Length:',sig_len,'samples')
print('Signal Duration:' , sig_dur , 'sec')
print(gbreak)
print('')

#divide into slices of 20 sec, with overlap of 2 sec
slice_len = 20*BASIC_SRATE
overlap = 2 * BASIC_SRATE

count = 0
istart = 0
iend = sig_len
while(istart<iend):
    ff,tt = istart , istart+slice_len
    ff,tt = rec.get_true_range(ff,tt)
    this_slice, this_list , this_pf , this_pt = rec.get_signal_slice_fd(ff,tt)
    
    count+=1
    if len(this_list)>0:
        temp = '[' + str(this_list[0])+':'+ str(this_list[-1]) + ']'
    
    else:
        temp = "~ no beats found in this interval"
    print('['+ str(count)+']','Interval:',round(ff/BASIC_SRATE,2),':',round(tt/BASIC_SRATE , 2), '=',round(len(this_slice)/BASIC_SRATE,2),'sec',' Samples:[',this_pf,':', this_pt,']',
          '\tNos Beats:', this_list.shape, temp )
    
    plot_beat(this_slice , this_list , this_pf , this_pt , this_pt - overlap,show_label_ticks = False , show_custom_labels = True , show_custom_vline = True,w = 0.012 , h=4,r = 0.5)
    istart+=slice_len-overlap

## Beat Representation

1. fixed length  - fixed number of samples from either side of R-peak

2. Variable length - based on the occurrance of previous and next R-peak

1. Fixed Length Represenation

In fixed length representation ,we decide the 'fixed number of samples to be taken on the either side of the R-peak

In [ ]:
# Fixed Length
# beat to be represented
bindex = 1852
# 1. Longer Representation - 1.3 sec on left and 1.5 sec on right
samp_left , samp_right = int(1.3 * BASIC_SRATE),int(1.5 * BASIC_SRATE)

signal_slice , signal_list , pf , pt = rec.get_signal_slice_biF(bindex,samp_left,samp_right)
plot_beat(signal_slice , signal_list , pf , pt , rec.iRpeaks[bindex],show_label_ticks = False , show_custom_labels = True , show_custom_vline = False, w = 0.012 , h=4 , r=0.5)

#2. Shorter Representation - 0.3 sec on left and 0.5 sec on right
 
samp_left , samp_right = int(0.3 * BASIC_SRATE),int(0.5* BASIC_SRATE)

signal_slice , signal_list , pf,pt = rec.get_signal_slice_biF(bindex,samp_left,samp_right)
plot_beat(signal_slice,signal_list,pf,pt,rec.iRpeaks[bindex],show_label_ticks = False , show_custom_labels = True,w = 0.012,h = 4,r = 0.5)

print(gbreak)
print('\nBeat:',rec.dbname+'/'+rec.name+'/'+str(bindex),
     '\nMapped-Label:', rec.mLabels[bindex],
     '\nStandard-Label:', rec.iLabels[bindex]
     )

2. Variable length representation

In [ ]:
# Example  - Variable length representation

#beat to be represented
bindex = 27

# Longer Representation
prev_i , next_i , trim_left,trim_right = 2,3,0,0

signal_slice , signal_list , pf,pt = rec.get_signal_slice_biV(bindex,prev_i, next_i,trim_left, trim_right)
plot_beat(signal_slice, signal_list, pf, pt, rec.iRpeaks[bindex],
                show_label_ticks=False, show_custom_labels=True, show_custom_vline=False,
                w=0.012, h=4, r=0.5)

#Shorter Representation

prev_i, next_i,trim_left, trim_right = 0,1,-20,12

signal_slice, signal_list, pf, pt = rec.get_signal_slice_biV(bindex,prev_i, next_i,trim_left, trim_right)
plot_beat(signal_slice, signal_list, pf, pt, rec.iRpeaks[bindex],
                show_label_ticks=False, show_custom_labels=True, show_custom_vline=False,
                w=0.012, h=4, r=0.5)

#<<----------------------------------------------------------------------------------------------
print('Beat:',rec.dbname+'/'+rec.name+'/'+str(bindex),
     '\nMapped-Label:', rec.mLabels[bindex],
     '\nStandard-Label:', rec.iLabels[bindex]
     )